# Note
This script doesn't work

In [1]:
import aiida

# must execute it in the first place
aiida.load_profile()

In [2]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List, Float
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os

from aiida.engine import calcfunction, workfunction, submit, run
from aiida.orm import load_code, load_node
from ase import io
from tools import wait_for_node_finished

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [3]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os

home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
print(computer_local)
code = get_or_create_code('alamode.displace_pf',
                          computer_local,
                          '/home/max/Documents/aiida-test/alamode-aiida/alamode/displace.py')
code_displace_pf = code
code = get_or_create_code('alamode.displace_random',
                          computer_local,
                          '/home/max/Documents/aiida-test/alamode-aiida/alamode/displace.py')
code_displace_random = code

# You must setup remote code
# if you need a setup of intel fortran, 
# . ~/intel/oneapi/setvars.sh
# is necessary.
code_simulator_name = Str('alamode.pwx@mygarden1')

code_displace_random, code_displace_pf, code_simulator_name

localhost (localhost), pk: 2


(<Code: Remote code 'alamode.displace_random' on localhost, pk: 2385, uuid: c893e5e1-c9ed-4d33-91eb-b4c8c2848412>,
 <Code: Remote code 'alamode.displace_pf' on localhost, pk: 2384, uuid: 1ded5557-bd21-4ebc-b2b5-f524c5ab4e25>,
 <Str: uuid: 7a800e77-97fc-4bd3-b5f7-4a84244837d3 (unstored) value: alamode.pwx@mygarden1>)

In [4]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path

{'CWD': 'run32'}


'/home/max/Documents/aiida-alamode_public/example/run32'

In [5]:
from os.path import expanduser
from tools import NodeBank
# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)

In [6]:
g_cwd = nodebank.load("cwd")  # ファイル保存directory
g_superstructure = nodebank.load("superstructure")  # 使用する長周期構造StructureData
g_superstructure_file = nodebank.load(
    "superstructure_file")  # 使用する長周期構造ファイルSinglefileData
g_superstructure_filepath = nodebank.load(
    "superstructure_filepath")  # 使用する長周期構造ファイルabspath
g_file_format = nodebank.load("structure_org_format")  # lammpsの位置フォーマット
g_file_style = nodebank.load("structure_org_style")  # lammpsの位置フォーマットのオプション
g_file_format, g_file_style, g_superstructure_file

ValueError: failed to find key=superstructure_file

## for QE

In [ ]:
key = "format"
g_format = nodebank.load_or_dump(key, Str("QE"))  # alamodeのファイルオプションの定義

In [ ]:
_dir = os.path.abspath('lammps_input/Si222')
if True:
    lammps_potentials = FolderData()
    for _p in ["Si.sw"]:
        abspath = os.path.join(_dir, _p)
        print(abspath, _p)
        lammps_potentials.put_object_from_file(abspath, path=_p)
else:
    _potential = []
    for _p in ["Si.sw"]:
        _potential.append(os.path.join(_dir, _p))
    lammps_potentials = List(list=_potential)

input_file_template = SinglefileData(os.path.join(_dir, "in.sw"))

In [ ]:
input_file_template.get_content()

In [ ]:
g_mag = nodebank.load_or_dump("mag", Float(0.01))

In [ ]:
g_pattern_files = nodebank.load("harmonic_pattern_folder")  # 作成したpattern files
g_pattern_files

In [ ]:
g_pattern_files.list_object_names()

In [ ]:
g_pattern_files.get_object_content('harmonic.pattern_HARMONIC')

In [ ]:
g_prefix = nodebank.load('prefix')
g_prefix

In [ ]:
g_action = "random"

if g_action == "pf":

    #codename = "alamode.displace_pf@tutor"
    #code = Code.get_from_string(codename)
    code = code_displace_pf

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = g_cwd
    builder.prefix = g_prefix
    builder.structure_org = g_superstructure_file
    builder.mag = g_mag
    builder.pattern_files = g_pattern_files

    g_displacefuture = nodebank.load("displace_pf")
    if g_displacefuture is None:
        g_displacefuture = submit(builder)
        print(g_displacefuture)
        wait_for_node_finished(g_displacefuture)
        if g_displacefuture.is_finished_ok:
            nodebank.dump("displace_pf", g_displacefuture)

elif g_action == "random":

    g_num_disp = nodebank.load_or_dump("num_disp", Int(5))

    #codename = "alamode.displace_random@tutor"
    #code = Code.get_from_string(codename)
    code = code_displace_random

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = g_cwd
    builder.prefix = g_prefix
    builder.structure_org = g_superstructure_file
    builder.mag = g_mag
    builder.num_disp = g_num_disp

    g_displacefuture = nodebank.load("displace_random")
    if g_displacefuture is None:
        g_displacefuture = submit(builder)
        print(g_displacefuture)
        wait_for_node_finished(g_displacefuture)
        if g_displacefuture.is_finished_ok:
            nodebank.dump("displace_random", g_displacefuture)
        else:
            raise
else:
    raise ValueError("unknown action")


In [ ]:
g_displacefuture


In [ ]:
g_displacefuture.outputs


In [ ]:
g_displace_result = g_displacefuture.outputs.result
g_displace_result


In [ ]:
g_dispfile_folder = g_displacefuture.outputs.dispfile_folder


In [ ]:
g_dispfile_folder.list_object_names()  # lammps data files


# run all the displacements

In [ ]:
from aiida.orm.nodes.data.upf import get_pseudos_from_structure

def _make_upf_locally( structure: StructureData, pseudos: str, cwd: str=None) -> dict:
    upflist = get_pseudos_from_structure(structure, pseudos)
    upfdic = {}
    for upfkey, upfvalue in upflist.items():
        #print(upfvalue.get_content())
        upf_filename = upfvalue.list_object_names()[0]
        upfdic[upfkey] = upf_filename
        if cwd is not None:
            target_path = os.path.join(cwd, upf_filename)
            with open(target_path,"w") as f:
                f.write(upfvalue.get_content())
            print(target_path,"is made.")
    return upfdic

In [ ]:
def make_qe_input(superstructure: StructureData, pseudos: (str,Str), QEinput = "qe.in"):
    params = {
            'calculation': 'scf',
             'tprnfor': True, 
             'tstress': True,
            'pseudo_dir': ".", 
            'ecutwfc': 3.,
            'ecutrho': 10.,
            'conv_thr': 5.e-6,
            'electron_maxstep': 100,
    }

    if isinstance(pseudos,Str):
        pseudos = pseudos.value
    pseudo = _make_upf_locally(superstructure, pseudos)
    kspacing  = 0.05
    koffset  = (1,1,1)
    with open(QEinput,"w") as f:
        write_espresso_in(f, superstructure.get_ase(), pseudopotentials =pseudo, 
                          kspacing= kspacing, koffset=koffset, 
                      **params)
    return True

In [ ]:
alldisp_lammps_WorkChain = WorkflowFactory("alamode.dispall_lammps")


In [ ]:
inputs = {"displace_result": g_displace_result,
          "data_folder": g_dispfile_folder,
          "input_file_template":  input_file_template,
          "code_string":  code_simulator_name,
          "cwd": g_cwd,
          "prefix": g_prefix,
          'potential_files': lammps_potentials}


In [ ]:
lammps_all = nodebank.load("all_lammps_harmonic")
if lammps_all is None:
    lammps_all = submit(alldisp_lammps_WorkChain, **inputs)
    wait_for_node_finished(lammps_all, 5)
    print(lammps_all.is_finished_ok)


In [ ]:
lammps_all


In [ ]:
lammps_all.outputs.dfset


# extract

In [ ]:
g_cwd


In [ ]:
from aiida.plugins import DataFactory, WorkflowFactory
extractworkchain = WorkflowFactory("alamode.extract")

g_target_file = lammps_all.outputs.dfset
g_norder = nodebank.load("norder")


In [ ]:
g_target_file


In [ ]:
g_superstructure_file


In [ ]:
g_superstructure_file


In [ ]:
inputs = {'format': g_format,
          'structure_org': g_superstructure_filepath,
          # 'target_file': g_pwx_all_future.outputs.pwx_output,
          'target_file': g_target_file,
          'cwd': g_cwd,
          'prefix': g_prefix
          # "norder": g_norder
          # 'output_filename' : g_DFSET_filename
          }
print(inputs)


In [ ]:
from tools import wait_for_node_finished


In [ ]:
g_alamodeextract_future = run(extractworkchain, **inputs)


In [ ]:
g_alamodeextract_future = submit(extractworkchain, **inputs)
print(g_alamodeextract_future)
wait_for_node_finished(g_alamodeextract_future)
print(g_alamodeextract_future.is_finished_ok)


In [ ]:
nodebank.dump("harmic_extract", g_alamodeextract_future)
